In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.375
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from geocat.comp import eofunc_eofs, eofunc_pcs
from datetime import datetime
import warnings
from matplotlib.patches import Rectangle
import pdo_functions
import importlib
importlib.reload(pdo_functions)
import random
import numba
import statsmodels.api as sm

In [ ]:
# Open CAPE data
cape_1940s = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_cape/ERA5_cape/1940s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19400101-19491231.nc')['CAPE'][184*10:]
cape_1950s = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_cape/ERA5_cape/1950s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19500101-19591231.nc')['CAPE'][184*10:]
cape_1960s = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_cape/ERA5_cape/1960s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19600101-19691231.nc')['CAPE'][184*10:]
cape_1970s = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_cape/ERA5_cape/1970s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19700101-19791231.nc')['CAPE'][184*10:]
cape_1980s = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_cape/ERA5_cape/1980s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19800101-19891231.nc')['CAPE']
cape_1990s = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_cape/ERA5_cape/1990s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19900101-19991231.nc')['CAPE']
cape_2000s = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_cape/ERA5_cape/2000s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.20000101-20091231.nc')['CAPE']
cape_2010s = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_cape/ERA5_cape/2010s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.20100101-20191231.nc')['CAPE']
cape_2020s = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ERA5_cape/ERA5_cape/2020s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.20200101-20241130.nc')['CAPE']

In [ ]:
# Select the CONUS only
cape_1940s_conus = cape_1940s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_1950s_conus = cape_1950s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_1960s_conus = cape_1960s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_1970s_conus = cape_1970s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_1980s_conus = cape_1980s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_1990s_conus = cape_1990s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_2000s_conus = cape_2000s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_2010s_conus = cape_2010s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_2020s_conus = cape_2020s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))

In [ ]:
# Concatenate time periods
cape_conus = xr.concat([cape_1940s_conus, cape_1950s_conus, cape_1960s_conus, cape_1970s_conus, cape_1980s_conus, \
                        cape_1990s_conus, cape_2000s_conus, cape_2010s_conus, cape_2020s_conus], dim='time')

In [ ]:
# Select MAM and resample to monthly means
cape_conus_mam = cape_conus.sel(time=cape_conus.time.dt.month.isin([3, 4, 5])).resample(time='1M').mean().dropna(dim='time')

In [ ]:
def detrend_dim(da, dim, deg):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit

In [ ]:
# degree 2 polynomial detrend
cape_conus_mam_detrend = detrend_dim(cape_conus_mam, 'time', 2)

In [ ]:
# Remove the seasonal cycle
cape_conus_mam_anoms = cape_conus_mam_detrend.groupby('time.month') - cape_conus_mam_detrend.groupby('time.month').mean()

In [ ]:
# Open SST data and compute the PDO and Nino3.4
sst = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/HadISST_sst.nc')['sst']

pdo = pdo_functions.pdo_from_hadisst(sst, 1980, 2010)

nino_34 = pdo_functions.calc_nino_34_timeseries(sst, False, 1980, 2010)

In [ ]:
# Standardize Nino3.4
nino_34_std = (nino_34 - nino_34.mean()) / nino_34.std()

In [ ]:
# Convert pdo to xarray
pdo_xr = xr.DataArray(pdo, coords={'time':nino_34['time']}, dims=['time'])

# Subset only 1940-2024
pdo_1940_2024 = pdo_xr.sel(time=pdo_xr.time.dt.year.isin(np.arange(1940, 2025, 1)))
nino_34_1940_2024 = nino_34_std.sel(time=nino_34_std.time.dt.year.isin(np.arange(1940, 2025, 1)))

# only MAM
pdo_1940_2024_mam = pdo_1940_2024.sel(time=pdo_1940_2024.time.dt.month.isin([3, 4, 5]))
nino_34_1940_2024_mam = nino_34_1940_2024.sel(time=nino_34_1940_2024.time.dt.month.isin([3, 4, 5]))

In [ ]:
# Subset an eastern US region
cape_conus_mam_anoms_east = cape_conus_mam_anoms.sel(longitude=slice(pdo_functions.convert_longitudes(-101), pdo_functions.convert_longitudes(-83))).sel(latitude=slice(47, 31))

In [ ]:
# Visualize regional subset
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

bounds = np.arange(0, 1050, 50)

norm = c.BoundaryNorm(bounds, plt.get_cmap('Reds').N)

ax.coastlines(lw=0.25, color='xkcd:gunmetal')
ax.add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')
ax.spines['geo'].set_edgecolor('xkcd:gunmetal')
ax.spines['geo'].set_linewidth(0.25)

ax.pcolormesh(cape_conus['longitude'], cape_conus['latitude'], cape_conus_mam.mean(dim='time'), transform=ccrs.PlateCarree(), shading='auto', norm=norm, cmap='Reds')

ax.pcolormesh(cape_conus_mam_anoms_east['longitude'], cape_conus_mam_anoms_east['latitude'], cape_conus_mam_anoms_east[0], transform=ccrs.PlateCarree(), shading='auto')

rect = Rectangle((-101, 31), -83- -101, 47-31, transform=ccrs.PlateCarree(), facecolor='none', edgecolor='black', lw=2)

ax.add_patch(rect)

In [ ]:
# Area weight and take spatial mean
weights = np.cos(np.deg2rad(cape_conus_mam_anoms.latitude))

cape_conus_mam_anoms_east_mean = cape_conus_mam_anoms_east.weighted(weights).mean(dim=('latitude', 'longitude'))

In [ ]:
np.arange(0, 255, 1)[pos_pdo_neg_cape]

In [ ]:
pdo_1940_2024_mam.values > 0

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

pos_pdo_neg_cape = (pdo_1940_2024_mam.values > 0) & (cape_conus_mam_anoms_east_mean.values < 0)
neg_pdo_pos_cape = (pdo_1940_2024_mam.values < 0) & (cape_conus_mam_anoms_east_mean.values > 0)
everything_else_pos = (pdo_1940_2024_mam.values > 0) & (cape_conus_mam_anoms_east_mean.values > 0)
everything_else_neg = (pdo_1940_2024_mam.values < 0) & (cape_conus_mam_anoms_east_mean.values < 0)

ax.bar(np.arange(1940, 2025, 1/3)[pos_pdo_neg_cape], cape_conus_mam_anoms_east_mean[pos_pdo_neg_cape], color='xkcd:carmine', label='(+) PDO & (-) CAPE', width=0.3)
ax.bar(np.arange(1940, 2025, 1/3)[neg_pdo_pos_cape], cape_conus_mam_anoms_east_mean[neg_pdo_pos_cape], color='xkcd:sapphire', label='(-) PDO & (+) CAPE', width=0.3)
ax.bar(np.arange(1940, 2025, 1/3)[everything_else_pos], cape_conus_mam_anoms_east_mean[everything_else_pos], color='xkcd:cement', width=0.3, label='everything else')
ax.bar(np.arange(1940, 2025, 1/3)[everything_else_neg], cape_conus_mam_anoms_east_mean[everything_else_neg], color='xkcd:cement', width=0.3)

ax.legend(frameon=False, fontsize=15, loc='upper left', handleheight=0.2, labelspacing=0.25)

ax.set_ylim(-600, 600)
ax.tick_params(labelsize=15)
ax.set_xlabel('time', size=15)
ax.set_ylabel('J kg$^{-1}$', size=15)

ax.set_title('1940-2024 MAM CAPE Anomalies', size=18)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Take annual means
pdo_ann = pdo_1940_2024_mam.groupby('time.year').mean()
cape_ann = cape_conus_mam_anoms_east_mean.groupby('time.year').mean()

pos_pdo_neg_cape = (pdo_ann.values > 0) & (cape_ann.values < 0)
neg_pdo_pos_cape = (pdo_ann.values < 0) & (cape_ann.values > 0)
everything_else_pos = (pdo_ann.values > 0) & (cape_ann.values > 0)
everything_else_neg = (pdo_ann.values < 0) & (cape_ann.values < 0)

ax.bar(np.arange(1940, 2025, 1)[pos_pdo_neg_cape], cape_ann[pos_pdo_neg_cape], color='xkcd:carmine', label='(+) PDO & (-) CAPE', width=0.95)
ax.bar(np.arange(1940, 2025, 1)[neg_pdo_pos_cape], cape_ann[neg_pdo_pos_cape], color='xkcd:sapphire', label='(-) PDO & (+) CAPE', width=0.95)
ax.bar(np.arange(1940, 2025, 1)[everything_else_pos], cape_ann[everything_else_pos], color='xkcd:cement', width=0.95, label='everything else')
ax.bar(np.arange(1940, 2025, 1)[everything_else_neg], cape_ann[everything_else_neg], color='xkcd:cement', width=0.95)

ax.legend(frameon=False, fontsize=15, loc='upper left', handleheight=0.2, labelspacing=0.25)

ax.set_ylim(-300, 300)
ax.tick_params(labelsize=15)
ax.set_xlabel('time', size=15)
ax.set_ylabel('J kg$^{-1}$', size=15)

ax.set_title('1940-2024 MAM CAPE Anomalies', size=18)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

pos_pdo = (pdo_1940_2024_mam.values > 0)
neg_pdo = (pdo_1940_2024_mam.values < 0)
pos_pdo_neg_cape = (pdo_1940_2024_mam.values > 0) & (cape_conus_mam_anoms_east_mean.values < 0)
neg_pdo_pos_cape = (pdo_1940_2024_mam.values < 0) & (cape_conus_mam_anoms_east_mean.values > 0)
everything_else_pos = (pdo_1940_2024_mam.values > 0) & (cape_conus_mam_anoms_east_mean.values > 0)
everything_else_neg = (pdo_1940_2024_mam.values < 0) & (cape_conus_mam_anoms_east_mean.values < 0)

ax.bar(np.arange(1940, 2025, 1/3)[pos_pdo], cape_conus_mam_anoms_east_mean[pos_pdo], color='xkcd:carmine', label='(+) PDO', width=0.3)
ax.bar(np.arange(1940, 2025, 1/3)[neg_pdo], cape_conus_mam_anoms_east_mean[neg_pdo], color='xkcd:sapphire', label='(-) PDO', width=0.3)

ax.legend(frameon=False, fontsize=15, loc='upper left', handleheight=0.2, labelspacing=0.25)

ax.set_ylim(-600, 600)
ax.tick_params(labelsize=15)
ax.set_xlabel('time', size=15)
ax.set_ylabel('J kg$^{-1}$', size=15)

ax.set_title('1940-2024 MAM CAPE Anomalies', size=18)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Take annual means
pdo_ann = pdo_1940_2024_mam.groupby('time.year').mean()
cape_ann = cape_conus_mam_anoms_east_mean.groupby('time.year').mean()

pos_pdo = (pdo_ann.values > 0)
neg_pdo = (pdo_ann.values < 0)
pos_pdo_neg_cape = (pdo_ann.values > 0) & (cape_ann.values < 0)
neg_pdo_pos_cape = (pdo_ann.values < 0) & (cape_ann.values > 0)
everything_else_pos = (pdo_ann.values > 0) & (cape_ann.values > 0)
everything_else_neg = (pdo_ann.values < 0) & (cape_ann.values < 0)

ax.bar(np.arange(1940, 2025, 1)[pos_pdo], cape_ann[pos_pdo], color='xkcd:carmine', label='(+) PDO', width=0.95)
ax.bar(np.arange(1940, 2025, 1)[neg_pdo], cape_ann[neg_pdo], color='xkcd:sapphire', label='(-) PDO', width=0.95)

ax.legend(frameon=False, fontsize=15, loc='upper left', handleheight=0.2, labelspacing=0.25)

ax.set_ylim(-300, 300)
ax.tick_params(labelsize=15)
ax.set_xlabel('time', size=15)
ax.set_ylabel('J kg$^{-1}$', size=15)

ax.set_title('1940-2024 MAM CAPE Anomalies', size=18)